In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

print(tf.__version__)

2.17.0


In [12]:
#   Get data directory
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data\\stack_overflow_16k\\')
test_dir = os.path.join(data_dir, "test")
train_dir = os.path.join(data_dir, "train")

#   Print example
with open(os.path.join(data_dir, "test\\csharp\\0.txt")) as f:
    print(f.read())

"how to specify the connection string if the excel file name contains white space?using blank string connstring = ""provider=microsoft.ace.oledb.12.0;data source=d:\data\[proj_resource details 20110118.xlsx];extended properties=excel 12.0"";...i mentioned [ ] still it is throwing exception.how can i solve this problem..plz mention the correct path"



In [ ]:
#   Create train/validation split
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    data_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

#   Print examples
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(3):
    print("Review", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])

print("Where label 0 corresponds to", raw_train_ds.class_names[0], "and ",
      "label 1 corresponds to", raw_train_ds.class_names[1])